In [1]:
from src.utilities_rejection import RejectionReasonLabeling, label_reasons, label_reasons_column
from src.data_local import MergedDataPreprocessing
from src.lstm_encoder import LSTMEmbedding
from src.model_train import encode_label
from cashe import get_cashed_input
import numpy as np
import time

In [24]:
df_original = get_cashed_input()

df_original = df_original[['VISIT_ID','CREATION_DATE','NOTES', 'OASIS_IOS_DESCRIPTION', 'SERVICE_TYPE','UNIT', 'UNIT_TYPE','TIMES','PER', 'PROVIDER_DEPARTMENT','ICD10','Chief_Complaint','OUTCOME']]

In [25]:
preprocessing = MergedDataPreprocessing(df_original)
preprocessing.fast_icd_encode()

df_train, df_test = preprocessing.train_test_split_time()

In [28]:
train_columns = list(df_train.columns)[3:]

X_train = df_train[train_columns[:-3]];  y_train = df_train[train_columns[-1:]]
X_test = df_test[train_columns[:-3]];  y_test = df_test[train_columns[-1:]]

y_train['OUTCOME'] = encode_label(y_train.OUTCOME.tolist()); y_test['OUTCOME'] = encode_label(y_test.OUTCOME.tolist())

C:\Users\omar.mounir\AppData\Local\Temp\ipykernel_9080\788751095.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_train['OUTCOME'] = encode_label(y_train.OUTCOME.tolist()); y_test['OUTCOME'] = encode_label(y_test.OUTCOME.tolist())


In [29]:
y_train

,OUTCOME
242103,1
235221,1
235220,0
235219,0
235287,1
...,...
300424,0
300425,0
300423,0
300422,0


In [30]:
lstm_embedding = LSTMEmbedding()
X_train_processed = lstm_embedding.preprocess_data(X_train)

In [22]:
### medical step adding to LSTM failed
#outcomes_train = label_reasons_column(y_train);  outcomes_test  = label_reasons_column(y_test)
#y_train = y_train.copy();  y_test = y_test.copy()
#y_train = [1 if elem == 2 else 0 for elem in outcomes_train]; y_test  = [1 if elem == 2 else 0 for elem in outcomes_test]

In [31]:
print('Training Model')
lstm_embedding.train_model(X_train_processed, y_train,epochs=10,batch_size=64)
## no big difference since epoch 10, only 10 would make it

print('Evaluating Model')
accuracy = lstm_embedding.evaluate_model(X_test, y_test)

print('Model Accuracy on test data: ',accuracy)

Training Model
Epoch 1/10
3694/3694 ━━━━━━━━━━━━━━━━━━━━ 21s 5ms/step - accuracy: 0.5546 - loss: 0.6807
Epoch 2/10
3694/3694 ━━━━━━━━━━━━━━━━━━━━ 19s 5ms/step - accuracy: 0.5949 - loss: 0.6539
Epoch 3/10
3694/3694 ━━━━━━━━━━━━━━━━━━━━ 19s 5ms/step - accuracy: 0.6134 - loss: 0.6427
Epoch 4/10
3694/3694 ━━━━━━━━━━━━━━━━━━━━ 19s 5ms/step - accuracy: 0.6237 - loss: 0.6360
Epoch 5/10
3694/3694 ━━━━━━━━━━━━━━━━━━━━ 20s 5ms/step - accuracy: 0.6308 - loss: 0.6326
Epoch 6/10
3694/3694 ━━━━━━━━━━━━━━━━━━━━ 19s 5ms/step - accuracy: 0.6335 - loss: 0.6303
Epoch 7/10
3694/3694 ━━━━━━━━━━━━━━━━━━━━ 20s 5ms/step - accuracy: 0.6327 - loss: 0.6299
Epoch 8/10
3694/3694 ━━━━━━━━━━━━━━━━━━━━ 19s 5ms/step - accuracy: 0.6369 - loss: 0.6272
Epoch 9/10
3694/3694 ━━━━━━━━━━━━━━━━━━━━ 19s 5ms/step - accuracy: 0.6381 - loss: 0.6268
Epoch 10/10
3694/3694 ━━━━━━━━━━━━━━━━━━━━ 19s 5ms/step - accuracy: 0.6405 - loss: 0.6247
Evaluating Model
1847/1847 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.6181 - loss: 0.6445


In [32]:
lstm_embedding.input_shape

(16, 1)

## Test Results

In [33]:
X_test_vectorized = lstm_embedding.embedding_vector(X_test[:3],reload_model=False)
X_test_vectorized

array([[ 1, 17,  4,  3, 10,  3, 15, 14,  7,  1,  4, 13,  2,  6,  1,  7],
       [ 1, 17,  4,  3, 10,  3, 15, 14,  7,  1,  4, 13,  2,  6,  1,  7],
       [ 1, 17,  4,  3, 10,  3, 15, 14,  7,  1,  4, 13,  2,  6,  1,  7]])

In [34]:
start_time = time.time()
lstm_encoder = LSTMEmbedding()
lstm_encoder.load_model()
X_test_vectorized = lstm_embedding.embedding_vector(X_test[:10],reload_model=True)

end_time = time.time()
execution_time = end_time - start_time
print("Execution time: {:.2f} seconds".format(execution_time))

Execution time: 0.55 seconds


In [35]:
X_test_vectorized

array([[ 14, 476, 239,  30, 335, 476, 335, 240,  81, 239,  30,  44,  14,
         24, 239,  81],
       [239,  81, 239,  30, 335, 476, 335, 240,  81, 239,  30,  44,  14,
         24, 239,  81],
       [239,  81, 239,  30, 335, 476, 335, 240,  81, 239,  30,  44,  14,
         24, 239,  81],
       [239,  81, 239,  30, 335, 476, 335, 240,  81, 239,  30,  44,  14,
         24, 239,  81],
       [ 14, 240, 239,  30, 335, 476, 335, 240,  81, 239,  30,  44,  14,
         24, 239,  81],
       [ 14, 476, 239,  30, 335, 476, 335, 240,  81, 239,  30,  44,  14,
         24, 239,  81],
       [356, 381, 356,  61, 476,  56,  14,  24,  81, 239,  30,  44,  14,
         24, 239,  81],
       [356, 381, 356,  61, 476,  56,  14,  24,  81, 239,  30,  44,  14,
         24, 239,  81],
       [356, 381, 356,  61, 476,  56,  14,  24,  81, 239,  30,  44,  14,
         24, 239,  81],
       [356, 381, 356,  61, 476,  56,  14,  24,  81, 239,  30,  44,  14,
         24, 239,  81]])

In [36]:
def vectorize(COUNT_VALS):
      preds = lstm_encoder.embedding_vector(X_test[:COUNT_VALS],reload_model=True)
      return

def calculate_excecution():
  exs = []
  COUNT_VALS = 50
  for i in range(10):
    start_time = time.time()

    vectorize(COUNT_VALS)

    end_time = time.time()

    execution_time = end_time - start_time
    exs.append(execution_time)

  average = np.array(exs).mean()
  return average/COUNT_VALS

In [37]:
calculate_excecution()

0.001843207836151123

In [38]:
X_test

,OASIS_IOS_DESCRIPTION,SERVICE_TYPE,UNIT,UNIT_TYPE,TIMES,PER,PROVIDER_DEPARTMENT
296669,TEGRETOL CR 200MG SCORED TAB,medication-codes,1,TAB,2,TWO TIMES DAILY,NEUROLOGY SERVICES مخ واعصاب
298738,COMPUTERIZED DIGITAL EEG\,procedures,None,None,None,None,NEUROLOGY SERVICES مخ واعصاب
298739,EXPERT CONS.,services,None,None,None,None,NEUROLOGY SERVICES مخ واعصاب
302638,EXPERT CONS.,services,None,None,None,None,NEUROLOGY SERVICES مخ واعصاب
302639,MRI PITUARY GLAND WITH CONTRAST,imaging,None,None,None,None,NEUROLOGY SERVICES مخ واعصاب
...,...,...,...,...,...,...,...
352722,CLARA 10MG TABLET,medication-codes,1,TAB,1,ONCE DAILY,EMERGENCY ROOM SERVICES - طوارى
352721,AZI-ONCE,medication-codes,1.2,CAP,1,ONCE DAILY,EMERGENCY ROOM SERVICES - طوارى
352720,KAFOSED SYRUP 15MG-5ML,medication-codes,1,BOT,1,IF NEEDED,EMERGENCY ROOM SERVICES - طوارى
352719,INTRAVENOUS INJECTION(INJ.ROOM,services,None,None,None,None,EMERGENCY ROOM SERVICES - طوارى
